In [1]:
#time in traffic value is in seconds text is in minutes

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from census import Census
import gmaps
import requests
import json
from pprint import pprint
from config import (api_key, gkey)
c = Census(api_key, year=2013)

In [3]:
#testing variable so im not running long code
#start_loc = ["Alameda,CA","Solano","Sonoma"]
start_loc = ["Alameda,CA","Contra Costa,CA","Marin","Napa","San Francisco","San Mateo","Santa Clara","Solano","Sonoma"]
end_loc = (list(reversed(start_loc)))
time_list = ["now"]
origin_Loc = []
destination_loc = []
trip_distance = []
trip_duration = []
traffic_duration = []
#target_url = (f'https://maps.googleapis.com/maps/api/distancematrix/json?origins=San Francisco,CA|Oakland,CA&destinations=San Jose,CA|Palo Alto,CA&departure_time=now&key={gkey}')

In [9]:
for time in time_list:

    for s_loc in start_loc:
        for e_loc in end_loc:
            
            target_url = (f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={s_loc}&destinations={e_loc},CA&departure_time={time}&key={gkey}')
            #target_url2 = (f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={e_loc}&destinations={s_loc},CA&departure_time={time}&key={gkey}')
            geo_data = requests.get(target_url).json()
            #geo_data2 = requests.get(target_url2).json()
            destination_loc.append(geo_data['destination_addresses'][0])
            origin_Loc.append(geo_data['origin_addresses'][0])
            trip_distance.append(geo_data['rows'][0]['elements'][0]['distance']['value'])
            traffic_duration.append(geo_data['rows'][0]['elements'][0]['duration_in_traffic']['value'])
            trip_duration.append(geo_data['rows'][0]['elements'][0]['duration']['value'])
            #pprint(geo_data)
      

{'destination_addresses': ['Sonoma, CA 95476, USA'],
 'origin_addresses': ['Alameda, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '84.9 km', 'value': 84929},
                         'duration': {'text': '1 hour 3 mins', 'value': 3786},
                         'duration_in_traffic': {'text': '59 mins',
                                                 'value': 3516},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Solano County, CA, USA'],
 'origin_addresses': ['Alameda, CA, USA'],
 'rows': [{'elements': [{'distance': {'text': '92.0 km', 'value': 91968},
                         'duration': {'text': '1 hour 6 mins', 'value': 3962},
                         'duration_in_traffic': {'text': '1 hour 2 mins',
                                                 'value': 3712},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Alameda, CA, USA'],
 'origin_addresses': ['Alameda, CA, USA'],
 'rows': [{'el

In [10]:
#converting units into minutes/miles
#minute_traffic = (traffic_duration-trip_duration)/60
traffic_duration = [x/60 for x in traffic_duration]
trip_duration  = [x/60 for x in trip_duration]
minute_traffic = [x-y for  x,y in zip(traffic_duration,trip_duration)]
#miles_distance = trip_distance/1,609.344
miles_distance = [z* 0.00062137 for z in trip_distance]
print(miles_distance)

[52.77233273, 57.14615616, 0.0, 39.22398125, 0.0, 56.9672016, 0.0, 38.8604798, 52.33675236, 52.77233273, 57.14615616, 0.0, 39.22398125, 0.0, 56.9672016, 0.0, 38.8604798, 52.33675236]


In [15]:
traffic_dic = {"Origin":origin_Loc,
               "Destination":destination_loc,
               "Distance(Miles)":miles_distance,
               "Trip length(minutes)":trip_duration,
               "Trip in traffic(minutes)":traffic_duration,
               "Traffic(minutes)":minute_traffic
              }
traffic_df = pd.DataFrame(traffic_dic).round(2)
traffic_df.head(10)

,Origin,Destination,Distance(Miles),Trip length(minutes),Trip in traffic(minutes),Traffic(minutes)
0,"Alameda, CA, USA","Sonoma, CA 95476, USA",52.77,0.02,0.02,-0.0
1,"Alameda, CA, USA","Solano County, CA, USA",57.15,0.02,0.02,-0.0
2,"Alameda, CA, USA","Alameda, CA, USA",0.00,0.00,0.00,0.0
3,"Solano County, CA, USA","Sonoma, CA 95476, USA",39.22,0.01,0.01,-0.0
4,"Solano County, CA, USA","Solano County, CA, USA",0.00,0.00,0.00,0.0
5,"Solano County, CA, USA","Alameda, CA, USA",56.97,0.02,0.02,-0.0
6,"Sonoma, CA 95476, USA","Sonoma, CA 95476, USA",0.00,0.00,0.00,0.0
7,"Sonoma, CA 95476, USA","Solano County, CA, USA",38.86,0.01,0.01,-0.0
8,"Sonoma, CA 95476, USA","Alameda, CA, USA",52.34,0.02,0.02,-0.0
9,"Alameda, CA, USA","Sonoma, CA 95476, USA",52.77,63.10,58.60,-4.5


In [ ]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"),{'for':'county:*'})
census_pd = pd.DataFrame(census_data)
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})
# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]


cali= census_pd[census_pd['State']== '06']
cali.head()